In [3]:
from vosk import Model, KaldiRecognizer, SetLogLevel

In [4]:
model_path = "models/vosk-model-small-en-us-0.15"
audio_filename = "audio/speech_recognition_systems.wav"

In [5]:
model = Model(model_path)

ERROR (VoskAPI:Model():model.cc:122) Folder 'models/vosk-model-en-us-0.21' does not contain model files. Make sure you specified the model path properly in Model constructor. If you are not sure about relative path, use absolute path specification.


Exception: Failed to create a model